# Experiment 14: Localized vs Distributed Forcing

**Phase 4 - Optimizing Restoration Strategies**

## Background

Experiment 10c showed that active forcing enables recovery:
- Linear relationship: recovery ≈ 0.74 × |f| + 0.51
- 50% recovery requires |f| ≥ 0.10
- 88% recovery achievable with |f| = 0.50

**But this applied UNIFORM forcing to all cells.**

## Key Question

**Is targeted forcing more efficient than distributed forcing?**

Conservation resources are limited. Compare strategies:

1. **Uniform forcing**: Equal forcing on all cells (baseline)
2. **High-connectivity targeting**: Focus on cells with highest degree
3. **High-flow targeting**: Focus on cells receiving most moisture
4. **Random targeting**: Focus on random subset (control)
5. **Keystone targeting**: Focus on cells identified as critical in Experiment 12

For a fixed "budget" of total forcing, which strategy recovers more cells?

## Experimental Design

| Parameter | Value |
|-----------|-------|
| Total forcing budget | [-5.0, -10.0, -15.0, -25.0] (sum of |f| across cells) |
| Targeting strategies | 5 (uniform, high-connectivity, high-flow, random, keystone) |
| Runs per condition | 30 |
| **Total simulations** | 4 × 5 × 30 = **600** |

## Hypothesis

Targeted forcing on high-connectivity or high-flow cells will achieve **20-30% better recovery** than uniform distribution for the same total forcing budget.

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from pathlib import Path
from netCDF4 import Dataset
from dask.distributed import as_completed

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    GradientDrivenCoupling,
    get_dask_client,
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Data

In [ ]:
DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")

## 3. Experiment Configuration

In [ ]:
# Experiment 14: Localized vs Distributed Forcing

SWEEP_CONFIG = {
    # Forcing budget: total |f| summed across all cells
    # For 50 cells, budget=5.0 means average |f|=0.1 per cell
    'budget_values': [5.0, 10.0, 15.0, 25.0],
    
    # Targeting strategies
    'strategies': ['uniform', 'high_connectivity', 'high_flow', 'random', 'concentrated'],
    
    # Runs per condition
    'n_runs_per_condition': 30,
    
    # Network parameters
    'n_cells': 50,
    'min_flow': 1.0,
    'barrier_height': 0.2,
    
    # Two-phase simulation parameters
    'cascade_duration': 200,
    'recovery_duration': 800,
    'dt': 0.5,
    'cascade_sigma': 0.06,
    'cascade_alpha': 1.5,
    'recovery_sigma': 0.04,
    'recovery_alpha': 2.0,
    
    # For targeted strategies: what fraction of cells receive forcing?
    'target_fraction': 0.3,  # 30% of cells (15 out of 50)
    
    # Seeds
    'base_seed': 42,
}

n_budgets = len(SWEEP_CONFIG['budget_values'])
n_strategies = len(SWEEP_CONFIG['strategies'])
n_runs = SWEEP_CONFIG['n_runs_per_condition']
total_sims = n_budgets * n_strategies * n_runs

print("=" * 60)
print("EXPERIMENT 14: LOCALIZED vs DISTRIBUTED FORCING")
print("=" * 60)
print(f"Budget values: {SWEEP_CONFIG['budget_values']}")
print(f"Strategies: {SWEEP_CONFIG['strategies']}")
print(f"Runs per condition: {n_runs}")
print(f"Total simulations: {total_sims}")
print(f"Target fraction for focused strategies: {SWEEP_CONFIG['target_fraction']:.0%}")
print(f"\nEstimated runtime: ~{total_sims * 2 / 60:.0f} minutes on 14 workers")

## 4. Network Creation with Cell Rankings

In [ ]:
def create_amazon_network_with_rankings(data, config, seed=42):
    """
    Create Amazon network and return cell rankings for targeting.
    
    Returns:
        network: EnergyConstrainedNetwork
        cell_info: DataFrame with cell properties and rankings
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    n_cells = config['n_cells']
    min_flow = config['min_flow']
    barrier_height = config['barrier_height']
    
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    cell_info = []
    
    # Add elements and calculate connectivity
    in_degree = np.zeros(n_cells)
    out_degree = np.zeros(n_cells)
    in_flow = np.zeros(n_cells)
    out_flow = np.zeros(n_cells)
    
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=barrier_height,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add couplings
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    out_degree[i] += 1
                    in_degree[j] += 1
                    out_flow[i] += flow
                    in_flow[j] += flow
    
    # Build cell info
    for i in range(n_cells):
        cell_info.append({
            'cell_id': i,
            'total_degree': in_degree[i] + out_degree[i],
            'in_flow': in_flow[i],
            'total_flow': in_flow[i] + out_flow[i],
        })
    
    cell_df = pd.DataFrame(cell_info)
    
    # Add rankings
    cell_df['connectivity_rank'] = cell_df['total_degree'].rank(ascending=False)
    cell_df['flow_rank'] = cell_df['in_flow'].rank(ascending=False)
    
    return net, cell_df, top_indices


# Create network
network, cell_info, selected_cells = create_amazon_network_with_rankings(
    amazon_data, SWEEP_CONFIG
)

print(f"Network: {network.n_elements} nodes, {network.number_of_edges()} edges")
print(f"\nCell rankings:")
print(cell_info.sort_values('connectivity_rank').head(10))

In [ ]:
def generate_forcing_pattern(cell_info, strategy, budget, target_fraction, seed=42):
    """
    Generate forcing pattern for each cell based on strategy.
    
    Returns:
        forcing_per_cell: array of forcing values (negative = restoration)
    """
    np.random.seed(seed)
    n_cells = len(cell_info)
    n_targets = int(n_cells * target_fraction)
    
    if strategy == 'uniform':
        # Equal forcing on all cells
        forcing_per_cell = np.ones(n_cells) * (-budget / n_cells)
        
    elif strategy == 'high_connectivity':
        # Focus on highest connectivity cells
        forcing_per_cell = np.zeros(n_cells)
        target_cells = cell_info.nsmallest(n_targets, 'connectivity_rank')['cell_id'].values
        forcing_per_cell[target_cells] = -budget / n_targets
        
    elif strategy == 'high_flow':
        # Focus on highest in-flow cells (moisture sinks)
        forcing_per_cell = np.zeros(n_cells)
        target_cells = cell_info.nsmallest(n_targets, 'flow_rank')['cell_id'].values
        forcing_per_cell[target_cells] = -budget / n_targets
        
    elif strategy == 'random':
        # Random selection of cells
        forcing_per_cell = np.zeros(n_cells)
        target_cells = np.random.choice(n_cells, n_targets, replace=False)
        forcing_per_cell[target_cells] = -budget / n_targets
        
    elif strategy == 'concentrated':
        # Extreme concentration: top 10% of cells
        forcing_per_cell = np.zeros(n_cells)
        n_concentrated = max(int(n_cells * 0.1), 3)  # At least 3 cells
        target_cells = cell_info.nsmallest(n_concentrated, 'connectivity_rank')['cell_id'].values
        forcing_per_cell[target_cells] = -budget / n_concentrated
        
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    return forcing_per_cell


# Test forcing patterns
print("Testing forcing patterns (budget=10.0):")
print(f"{'Strategy':<20} {'Min f':<10} {'Max f':<10} {'Non-zero cells':<15} {'Sum |f|':<10}")
print("-" * 65)
for strategy in SWEEP_CONFIG['strategies']:
    pattern = generate_forcing_pattern(
        cell_info, strategy, 10.0, SWEEP_CONFIG['target_fraction'], seed=42
    )
    n_nonzero = np.sum(pattern != 0)
    print(f"{strategy:<20} {pattern.min():<10.3f} {pattern.max():<10.3f} "
          f"{n_nonzero:<15} {np.abs(pattern).sum():<10.1f}")

## 5. Worker Function with Heterogeneous Forcing

In [ ]:
def run_targeted_forcing_experiment(args):
    """
    Worker function for targeted forcing experiment.
    
    Applies cell-specific forcing during recovery phase.
    """
    network_bytes, forcing_pattern, strategy, budget, config, seed = args
    
    import sys
    import numpy as np
    import pickle
    
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    from energy_constrained.solvers import energy_constrained_euler_maruyama, SolverResult
    
    # Reconstruct network
    network = pickle.loads(network_bytes)
    np.random.seed(seed)
    n_elements = network.n_elements
    
    # Prepare initial state
    y0 = network.get_initial_state(None)
    
    # === Phase 1: Cascade ===
    cascade_result = energy_constrained_euler_maruyama(
        f_extended=network.f_extended,
        y0=y0,
        t_span=(0, config['cascade_duration']),
        dt=config['dt'],
        sigma=config['cascade_sigma'] * np.ones(n_elements),
        alpha=config['cascade_alpha'] * np.ones(n_elements)
    )
    
    # Record state at cascade end
    cascade_end_y = cascade_result.y[-1].copy()
    cascade_end_state = cascade_result.x[-1].copy()
    tipped_at_cascade_end = cascade_end_state > 0
    n_tipped = np.sum(tipped_at_cascade_end)
    
    # === Phase 2: Recovery with heterogeneous forcing ===
    # Create modified dynamics with cell-specific forcing
    original_f = network.f_extended
    forcing_array = np.array(forcing_pattern)
    
    def f_with_heterogeneous_forcing(t, y):
        dydt = original_f(t, y)
        # Add forcing to state variables (first n_elements)
        dydt[:n_elements] += forcing_array
        return dydt
    
    recovery_result = energy_constrained_euler_maruyama(
        f_extended=f_with_heterogeneous_forcing,
        y0=cascade_end_y,
        t_span=(0, config['recovery_duration']),
        dt=config['dt'],
        sigma=config['recovery_sigma'] * np.ones(n_elements),
        alpha=config['recovery_alpha'] * np.ones(n_elements)
    )
    
    # Combine trajectories
    x_full = np.concatenate([cascade_result.x, recovery_result.x[1:]], axis=0)
    
    # === Analyze Recovery ===
    recovered = np.zeros(n_elements, dtype=bool)
    recovery_times = {}
    
    for i in range(n_elements):
        if tipped_at_cascade_end[i]:
            recovery_traj = recovery_result.x[:, i]
            recovery_crossings = np.where(recovery_traj < 0)[0]
            
            for cross_idx in recovery_crossings:
                if cross_idx + 10 < len(recovery_traj):
                    if np.all(recovery_traj[cross_idx:cross_idx+10] < 0):
                        recovered[i] = True
                        recovery_times[i] = cross_idx * config['dt']
                        break
                else:
                    if np.all(recovery_traj[cross_idx:] < 0):
                        recovered[i] = True
                        recovery_times[i] = cross_idx * config['dt']
                        break
    
    n_recovered = np.sum(recovered)
    recovery_fraction = n_recovered / n_tipped if n_tipped > 0 else np.nan
    
    # Additional metrics
    pct_time_tipped = np.mean(x_full > 0) * 100
    final_pct_tipped = np.mean(recovery_result.x[-1] > 0) * 100
    
    # Efficiency: recovery per unit forcing
    efficiency = recovery_fraction / budget if budget > 0 else np.nan
    
    return {
        'strategy': strategy,
        'budget': budget,
        'seed': seed,
        'recovery_fraction': recovery_fraction,
        'n_tipped_cascade': int(n_tipped),
        'n_recovered': int(n_recovered),
        'n_permanent_tips': int(n_tipped - n_recovered),
        'pct_tipped_cascade': float(n_tipped / n_elements * 100),
        'final_pct_tipped': final_pct_tipped,
        'pct_time_tipped': pct_time_tipped,
        'efficiency': efficiency,
        'mean_recovery_time': np.nanmean(list(recovery_times.values())) if recovery_times else np.nan,
    }

print("Worker function defined.")

## 6. Run Targeting Strategy Comparison

In [ ]:
# Serialize and scatter network
network_bytes = pickle.dumps(network)
print(f"Network serialized: {len(network_bytes) / 1024:.1f} KB")

network_future = client.scatter(network_bytes, broadcast=True)
print("Network broadcast to all workers")

# Build task arguments
print("\n" + "=" * 60)
print("EXPERIMENT 14: Starting Strategy Comparison")
print("=" * 60)
start_time = time.time()

task_args = []
for b_idx, budget in enumerate(SWEEP_CONFIG['budget_values']):
    for s_idx, strategy in enumerate(SWEEP_CONFIG['strategies']):
        for run_idx in range(SWEEP_CONFIG['n_runs_per_condition']):
            seed = SWEEP_CONFIG['base_seed'] + b_idx * 100000 + s_idx * 10000 + run_idx
            
            # Generate forcing pattern for this run
            # Use different seed for random strategy to get variety
            pattern_seed = seed if strategy == 'random' else 42
            forcing_pattern = generate_forcing_pattern(
                cell_info, strategy, budget, SWEEP_CONFIG['target_fraction'], pattern_seed
            )
            
            task_args.append((
                network_bytes, forcing_pattern.tolist(), strategy, budget, SWEEP_CONFIG, seed
            ))

print(f"Generated {len(task_args)} task arguments")

# Submit tasks
futures = client.map(run_targeted_forcing_experiment, task_args)
print(f"Submitted {len(futures)} tasks")

# Collect results
all_results = []
print("\nProgress:")
for i, future in enumerate(as_completed(futures)):
    result = future.result()
    all_results.append(result)
    
    if (i + 1) % 60 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed
        remaining = (len(futures) - i - 1) / rate
        print(f"  Completed {i+1}/{len(futures)} ({100*(i+1)/len(futures):.1f}%) "
              f"- {elapsed:.0f}s elapsed, ~{remaining:.0f}s remaining")

elapsed = time.time() - start_time
print(f"\n" + "=" * 60)
print(f"COMPLETE: {len(all_results)} simulations in {elapsed:.1f}s ({elapsed/60:.1f} min)")
print("=" * 60)

## 7. Results Aggregation

In [ ]:
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")

df = df.sort_values(['budget', 'strategy', 'seed']).reset_index(drop=True)
df.head(10)

In [ ]:
# Summary statistics by budget and strategy
summary = df.groupby(['budget', 'strategy']).agg({
    'recovery_fraction': ['mean', 'std', 'min', 'max'],
    'n_permanent_tips': ['mean', 'std'],
    'efficiency': ['mean', 'std'],
    'mean_recovery_time': 'mean',
}).round(4)

summary.columns = ['_'.join(col) for col in summary.columns]
summary = summary.reset_index()

print("=" * 90)
print("STRATEGY COMPARISON SUMMARY")
print("=" * 90)
print(summary[['budget', 'strategy', 'recovery_fraction_mean', 'recovery_fraction_std',
               'n_permanent_tips_mean', 'efficiency_mean']].to_string(index=False))

## 8. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Color palette for strategies
strategy_colors = {
    'uniform': 'gray',
    'high_connectivity': 'blue',
    'high_flow': 'green',
    'random': 'orange',
    'concentrated': 'red',
}

# Panel 1: Recovery vs Budget (by strategy)
ax = axes[0, 0]
for strategy in SWEEP_CONFIG['strategies']:
    subset = summary[summary['strategy'] == strategy]
    ax.errorbar(
        subset['budget'].values,
        subset['recovery_fraction_mean'].values,
        yerr=subset['recovery_fraction_std'].values,
        marker='o', capsize=4, linewidth=2, markersize=8,
        color=strategy_colors[strategy], label=strategy
    )
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.7)
ax.set_xlabel('Total Forcing Budget', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery vs Budget by Strategy', fontsize=14)
ax.legend(loc='lower right')
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)

# Panel 2: Efficiency (recovery per unit budget)
ax = axes[0, 1]
for strategy in SWEEP_CONFIG['strategies']:
    subset = summary[summary['strategy'] == strategy]
    ax.plot(
        subset['budget'].values,
        subset['efficiency_mean'].values,
        marker='s', linewidth=2, markersize=8,
        color=strategy_colors[strategy], label=strategy
    )
ax.set_xlabel('Total Forcing Budget', fontsize=12)
ax.set_ylabel('Efficiency (Recovery / Budget)', fontsize=12)
ax.set_title('Forcing Efficiency by Strategy', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Panel 3: Strategy Comparison Bar Chart (at budget=10)
ax = axes[1, 0]
budget_10 = summary[summary['budget'] == 10.0]
x_pos = range(len(budget_10))
bars = ax.bar(x_pos, budget_10['recovery_fraction_mean'],
              yerr=budget_10['recovery_fraction_std'],
              color=[strategy_colors[s] for s in budget_10['strategy']],
              capsize=4, edgecolor='black')
ax.set_xticks(x_pos)
ax.set_xticklabels(budget_10['strategy'], rotation=45, ha='right')
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Strategy Comparison at Budget=10', fontsize=14)
ax.axhline(budget_10[budget_10['strategy']=='uniform']['recovery_fraction_mean'].values[0],
           color='gray', linestyle='--', alpha=0.7, label='Uniform baseline')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Panel 4: Box plot of recovery by strategy (all budgets)
ax = axes[1, 1]
data_by_strategy = [df[df['strategy'] == s]['recovery_fraction'].values
                    for s in SWEEP_CONFIG['strategies']]
bp = ax.boxplot(data_by_strategy, labels=SWEEP_CONFIG['strategies'],
                patch_artist=True)

for patch, strategy in zip(bp['boxes'], SWEEP_CONFIG['strategies']):
    patch.set_facecolor(strategy_colors[strategy])
    patch.set_alpha(0.7)

ax.set_xticklabels(SWEEP_CONFIG['strategies'], rotation=45, ha='right')
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery Distribution by Strategy (all budgets)', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/workspace/data/exp14_strategy_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp14_strategy_comparison.png")

## 9. Strategy Comparison Analysis

In [ ]:
# Calculate improvement over uniform baseline
print("=" * 70)
print("IMPROVEMENT OVER UNIFORM BASELINE")
print("=" * 70)

improvement_results = []

for budget in SWEEP_CONFIG['budget_values']:
    print(f"\nBudget = {budget}:")
    uniform_rec = summary[(summary['budget'] == budget) & 
                          (summary['strategy'] == 'uniform')]['recovery_fraction_mean'].values[0]
    
    for strategy in SWEEP_CONFIG['strategies']:
        if strategy != 'uniform':
            strat_rec = summary[(summary['budget'] == budget) & 
                               (summary['strategy'] == strategy)]['recovery_fraction_mean'].values[0]
            improvement = (strat_rec - uniform_rec) / uniform_rec * 100 if uniform_rec > 0 else 0
            
            improvement_results.append({
                'budget': budget,
                'strategy': strategy,
                'recovery': strat_rec,
                'uniform_recovery': uniform_rec,
                'improvement_pct': improvement
            })
            
            print(f"  {strategy:<20}: {strat_rec:.3f} ({improvement:+.1f}% vs uniform)")

improvement_df = pd.DataFrame(improvement_results)

# Best strategy per budget
print("\n" + "=" * 70)
print("BEST STRATEGY BY BUDGET")
print("=" * 70)
for budget in SWEEP_CONFIG['budget_values']:
    budget_data = improvement_df[improvement_df['budget'] == budget]
    best = budget_data.loc[budget_data['improvement_pct'].idxmax()]
    print(f"Budget {budget}: {best['strategy']} ({best['improvement_pct']:+.1f}% improvement)")

In [ ]:
# Cost-effectiveness analysis
print("\n" + "=" * 70)
print("COST-EFFECTIVENESS ANALYSIS")
print("=" * 70)

# Find minimum budget to reach 50% recovery
print("\nMinimum budget for 50% recovery:")
for strategy in SWEEP_CONFIG['strategies']:
    strat_data = summary[summary['strategy'] == strategy].sort_values('budget')
    
    # Find crossing point
    above_50 = strat_data[strat_data['recovery_fraction_mean'] >= 0.5]
    if len(above_50) > 0:
        min_budget = above_50['budget'].min()
        print(f"  {strategy:<20}: budget = {min_budget:.1f}")
    else:
        print(f"  {strategy:<20}: not reached (max = {strat_data['recovery_fraction_mean'].max():.2f})")

## 10. Key Findings Summary

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 14: KEY FINDINGS")
print("=" * 70)

# Overall best strategy
overall_best = df.groupby('strategy')['recovery_fraction'].mean().idxmax()
overall_best_recovery = df.groupby('strategy')['recovery_fraction'].mean().max()

# Uniform baseline
uniform_recovery = df[df['strategy'] == 'uniform']['recovery_fraction'].mean()

# Most efficient
most_efficient = df.groupby('strategy')['efficiency'].mean().idxmax()
best_efficiency = df.groupby('strategy')['efficiency'].mean().max()

print(f"""
1. OVERALL BEST STRATEGY:
   {overall_best}
   Mean recovery: {overall_best_recovery:.1%}
   Improvement over uniform: {(overall_best_recovery - uniform_recovery) / uniform_recovery * 100:+.1f}%

2. MOST EFFICIENT STRATEGY:
   {most_efficient}
   Mean efficiency: {best_efficiency:.4f} recovery/unit budget

3. UNIFORM BASELINE:
   Mean recovery: {uniform_recovery:.1%}

4. STRATEGY RANKINGS (by mean recovery):
""")

rankings = df.groupby('strategy')['recovery_fraction'].mean().sort_values(ascending=False)
for i, (strategy, recovery) in enumerate(rankings.items()):
    diff = (recovery - uniform_recovery) / uniform_recovery * 100 if strategy != 'uniform' else 0
    print(f"   {i+1}. {strategy:<20}: {recovery:.1%} ({diff:+.1f}% vs uniform)")

# Policy implications
print(f"""
5. POLICY IMPLICATIONS:
""")

if overall_best != 'uniform':
    print(f"   - Targeted forcing ({overall_best}) is more effective than uniform distribution")
    improvement = (overall_best_recovery - uniform_recovery) / uniform_recovery * 100
    print(f"   - {improvement:.0f}% more recovery for the same total investment")
else:
    print("   - Uniform forcing is as effective as targeted approaches")
    print("   - No additional benefit from targeting specific cells")

## 11. Save Results

In [ ]:
df.to_csv('/workspace/data/experiment14_strategy_comparison.csv', index=False)
print(f"Full results saved to /workspace/data/experiment14_strategy_comparison.csv")

summary.to_csv('/workspace/data/experiment14_summary.csv', index=False)
print(f"Summary saved to /workspace/data/experiment14_summary.csv")

improvement_df.to_csv('/workspace/data/experiment14_improvements.csv', index=False)
print(f"Improvement analysis saved to /workspace/data/experiment14_improvements.csv")

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 14 COMPLETE")
print("=" * 70)

print(f"""
CONFIGURATION:
- Budget values: {SWEEP_CONFIG['budget_values']}
- Strategies: {SWEEP_CONFIG['strategies']}
- Runs per condition: {SWEEP_CONFIG['n_runs_per_condition']}
- Total simulations: {len(df)}
- Runtime: {elapsed:.1f}s ({elapsed/60:.1f} min)

FILES GENERATED:
- /workspace/data/experiment14_strategy_comparison.csv
- /workspace/data/experiment14_summary.csv
- /workspace/data/experiment14_improvements.csv
- /workspace/data/exp14_strategy_comparison.png

NEXT STEPS:
- Test hybrid strategies (mix of targeting approaches)
- Map optimal targets to geographic locations
- Cost-benefit analysis with real intervention costs
""")